In [36]:
from transformers import BertTokenizer
from transformers import BertConfig
from transformers import BertForMaskedLM
from datasets import load_dataset
from datasets import load_from_disk
from datasets import Dataset
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import datasets
import torch
import pandas as pd

In [37]:
torch.cuda.is_available()

True

In [38]:
max_length = 1024
vocab_size = 30
logging_dir = '../ModelLogs/Dumberta'
model_dir = '../models/Dumberta'

In [42]:
train_path = "../Datasets/Degree_tokenized_split_three_ways/sorted_train.csv"
test_path = "../Datasets/Degree_tokenized_split_three_ways/sorted_test.csv"
train_df = pd.read_csv(train_path,names=['Index','Labels', 'Sequence','Degree','Tokenized Sequence'],skiprows=1)
test_df = pd.read_csv(test_path,names=['Index','Labels', 'Sequence','Degree','Tokenized Sequence'],skiprows=1)
train_seqs, test_seqs = train_df['Sequence'], test_df['Sequence']
train_seqs.to_csv('../Datasets/Sequences_from_Degree_tokenized/train.csv', index=False)
test_seqs.to_csv('../Datasets/Sequences_from_Degree_tokenized/test.csv', index=False)

In [43]:
train_dataset = load_dataset('csv', data_files='../Datasets/Sequences_from_Degree_tokenized/train.csv')
validation_dataset = load_dataset('csv', data_files='../Datasets/Sequences_from_Degree_tokenized/test.csv')

Using custom data configuration default-85dea832c9d9a2c9


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/johnmf4/.cache/huggingface/datasets/csv/default-85dea832c9d9a2c9/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


Using custom data configuration default-5546172523b52439


0 tables [00:00, ? tables/s]

Dataset csv downloaded and prepared to /home/johnmf4/.cache/huggingface/datasets/csv/default-5546172523b52439/0.0.0/2dc6629a9ff6b5697d82c25b73731dd440507a69cbce8b425db50b751e8fcfd0. Subsequent calls will reuse this data.


In [41]:
%%time
train_dataset = load_dataset('text', data_files='../Datasets/UniRef90_Data/shrunken_train.txt')
validation_dataset = load_dataset('text', data_files='../Datasets/UniRef90_Data/uniref90_test.txt')

FileNotFoundError: [Errno 2] No such file or directory: '../Datasets/UniRef90_Data/shrunken_train.txt'

In [44]:
config = BertConfig(
    vocab_size=vocab_size,
    max_position_embeddings=max_length + 2,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1
)

In [45]:
tokenizer = BertTokenizer.from_pretrained("../models/Dumberta", do_lower_case=False )
model = BertForMaskedLM(config=config)

In [46]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [47]:
training_args = TrainingArguments(
    output_dir="../models/Dumberta",
    overwrite_output_dir=False,
    num_train_epochs=200,
    per_device_train_batch_size=60,
    eval_steps=10_000,
    evaluation_strategy="steps",
    metric_for_best_model="loss",
    save_steps=500,
    save_total_limit=100,
    prediction_loss_only=True,
    load_best_model_at_end=True,
    fp16=True,
    logging_dir="../ModelLogs/Dumberta"

)

In [49]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset
)

In [50]:
trainer.train()
trainer.save_model("../models/Dumberta")

KeyError: 0